In [375]:
#IMPORTING LIBRARY FOR HANDING DATAFRAME AND DATA VISUALIZATION
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [376]:
#READING THE CSV FILE USING THE PYTHON
crop_dataframe = pd.read_csv("crop dataset.csv")

                                          CROPS DATASET CLEANING


In [ ]:
#DISPLAYING THE DATAFRAME
crop_dataframe.head()

In [ ]:
#CHECKING THE NULL VALUE
crop_dataframe.isna().sum()

In [ ]:
#CHECK THE COUNT 
crop_dataframe.count()

In [380]:
#DROP THE ROW STOCK WHICH IS NOT USED FOR ANALYSIS
crop_dataframe.drop(crop_dataframe[crop_dataframe["Element"] == "Stocks"].index, axis=0, inplace=True)


In [ ]:
#CHECKING THE MISSING VALUE FOR THE PRODUCTION USING FLAG DESCRIPTION
crop_dataframe[crop_dataframe["Flag Description"]=="Missing value (data cannot exist, not applicable)"]

In [382]:
#DROP THE MISSING VALUE
crop_dataframe.drop(crop_dataframe[crop_dataframe["Flag Description"] == "Missing value (data cannot exist, not applicable)"].index, axis=0, inplace=True)

In [ ]:
#CHECK WHETHER ANY NULL VALUE IN ITEM CODE AND FILLED THE NULL VALUE IN ORIGINAL CSV FILE
crop_dataframe[crop_dataframe['Item Code (CPC)'].isna()]

In [ ]:
# AFTER DROPING AND FILLING THE VALUE AGAIN CHECKING THE NULL
crop_dataframe.isna().sum()

In [385]:
# FILLING NULL USING MODE
mode_area_harvested = crop_dataframe[crop_dataframe['Element'] == 'Area harvested']['Value'].mode()[0]
crop_dataframe.loc[(crop_dataframe['Element'] == 'Area harvested') & (crop_dataframe['Value'].isna()), 'Value'] = mode_area_harvested



In [386]:
# FILLING NULL USING MODE
mode_area_harvested = crop_dataframe[crop_dataframe['Element'] == 'Yield']['Value'].mode()[0]
crop_dataframe.loc[(crop_dataframe['Element'] == 'Yield') & (crop_dataframe['Value'].isna()), 'Value'] = mode_area_harvested

In [387]:
# FILLING NULL USING MODE
mode_area_harvested = crop_dataframe[crop_dataframe['Element'] == 'Production']['Value'].mode()[0]
crop_dataframe.loc[(crop_dataframe['Element'] == 'Production') & (crop_dataframe['Value'].isna()), 'Value'] = mode_area_harvested

In [ ]:
# FILLING THE REMAINING THE NULL VALUE
crop_dataframe["Value"].fillna(0,inplace = True)

In [389]:
#DROP THE UNWANTED COLUMNS WHICH IS NOT USED FOR ANALYSIS AND TRAING 
crop_dataframe.drop(columns="Note", inplace=True)
crop_dataframe.drop(columns="Year Code", inplace=True)
crop_dataframe.drop(columns="Flag", inplace=True)
crop_dataframe.drop(columns="Flag Description", inplace=True)


In [ ]:
#CHECKING THE UNIQUE VALUE IN ELEMENT FOR SEPARATE THE COLUMNS FOR PREDICTION
crop_dataframe["Element"].unique()

In [391]:
#SEPARATE THE COLUMNS USING THE LOOPS
elements = ['Area harvested', 'Yield',
       'Producing Animals/Slaughtered', 'Laying', 'Yield/Carcass Weight',
       'Milk Animals','Production']

for element in elements:
    Tranformed_Columns = crop_dataframe['Element'] == element
    crop_dataframe[f'{element}_Value'] = crop_dataframe['Value'].where(Tranformed_Columns, None)

In [ ]:
#DISPLAYING THE SEPARATED COLUMNS
crop_dataframe.head()

In [393]:
# RENAME THE COLUMNS WITH UNIT 
crop_dataframe = crop_dataframe.rename(columns={"Area harvested_Value": "Area_Harvested_in_Hectares"})
crop_dataframe = crop_dataframe.rename(columns={"Yield_Value": "Yield_Value in kg/ha(or)mg/Ar"})
crop_dataframe = crop_dataframe.rename(columns={"Production_Value": "Production in Tones"})
crop_dataframe = crop_dataframe.rename(columns={"Producing Animals/Slaughtered_Value": "Producing Animals/Slaughtered_Value in An"})
crop_dataframe = crop_dataframe.rename(columns={"Laying_Value": "Laying_Value in An"})
crop_dataframe = crop_dataframe.rename(columns={"Yield/Carcass Weight_Value": "Yield/Carcass Weight_Value in g/An"})
crop_dataframe = crop_dataframe.rename(columns={"Milk Animals_Value": "Milk Animals_Value in An"})


In [394]:
#DROPING THE COLUMNS WHICH IS USED BEFORE SEPERATION
crop_dataframe.drop("Element",axis = 1,inplace = True)
crop_dataframe.drop("Value",axis = 1,inplace = True)
crop_dataframe.drop("Unit",axis = 1,inplace = True)
crop_dataframe.drop("Element Code",axis = 1,inplace = True)


In [ ]:
#GROUPING THE COLUMNS BY YEAR TO MAKE THE PROPER DATAFRAME FOR PREDICTION AND ANALYSIS
grouped_crop_Dataframe = crop_dataframe.groupby(
    ['Domain','Area', 'Item','Item Code (CPC)','Year'], as_index=False
).agg({
    'Area_Harvested_in_Hectares': 'sum',  
    'Yield_Value in kg/ha(or)mg/Ar': 'sum',  
    'Producing Animals/Slaughtered_Value in An': 'sum',  
    'Laying_Value in An': 'sum',  
    'Yield/Carcass Weight_Value in g/An': 'sum',  
    'Milk Animals_Value in An': 'sum',  
    'Production in Tones': 'sum'  
})


grouped_crop_Dataframe.head()


In [ ]:
#AFTER GROUPING CHECKING THE COUNT
grouped_crop_Dataframe.count()


In [ ]:
#AFTER GROUPING CHECKING THE NULL AGAIN
grouped_crop_Dataframe.isna().sum()

In [ ]:
#CHECKING THE DATA TYPE 
grouped_crop_Dataframe.dtypes

In [448]:
#AFTER CHECKING DATATYPE THE ITEM CODE IS OBJECT BUT IT IS FLOAT REPLACE THE ONE WRONG OBJECT WITH FLOAT
grouped_crop_Dataframe['Item Code (CPC)']=grouped_crop_Dataframe['Item Code (CPC)'].replace('2351f','2351')

In [449]:
#CONVERTING THE ITEM CODE DATATYPE TO FLOAT
grouped_crop_Dataframe['Item Code (CPC)']=grouped_crop_Dataframe['Item Code (CPC)'].astype(float)

In [ ]:
#AFTER CONVERTING AGAIN CHECKING THE DATATYPE
grouped_crop_Dataframe.dtypes

In [451]:
#ROUND THE COLUMNS VALUE FOR GOOD TRAINING
columns_to_round = [
    "Area_Harvested_in_Hectares",
    "Yield_Value in kg/ha(or)mg/Ar",
    "Producing Animals/Slaughtered_Value in An",
    "Laying_Value in An",
    "Yield/Carcass Weight_Value in g/An",
    "Milk Animals_Value in An",
    "Production in Tones"
]

grouped_crop_Dataframe[columns_to_round] = grouped_crop_Dataframe[columns_to_round].round(1)




In [ ]:
##CHECKING THE TARGET PRODUCTION HAS ANY O VALUES
grouped_crop_Dataframe[grouped_crop_Dataframe["Production in Tones"]==0.0]

In [453]:
#DROPING THE ROW HAS PRODUCTION=0
grouped_crop_Dataframe = grouped_crop_Dataframe[grouped_crop_Dataframe["Production in Tones"] != 0.0]

In [ ]:
#CHECKING THE NULL AGAIN
grouped_crop_Dataframe.isna().sum()

In [ ]:
#DISPLAY THE CLEANED AND PREPROCESSED DATA FOR TRAINING THE MODEL
grouped_crop_Dataframe.head()

In [186]:
#CONVERTING THE CLEANED DATAFRAME TO CSV
grouped_crop_Dataframe.to_csv("CLEANED_CROP_DATASET.csv",index=False)

                                             EXPLORATORY DATA ANALYSIS

In [456]:
#IMPORTING LIBRARIES FOR EDA
import matplotlib.pyplot as plt
import seaborn as sns

UNIVARIENT ANALYSIS

In [ ]:
#CHECKING THE INFO FOR DATAFRAME
grouped_crop_Dataframe.info()

In [ ]:
#CHECKING THE DESCRIPTION FOR DATAFRAME
grouped_crop_Dataframe.describe()

In [ ]:
#CHECKING THE ALL YEAR UNIQUE
grouped_crop_Dataframe["Year"].unique()

In [ ]:
#COLUMNS NAME
grouped_crop_Dataframe.columns


BI-VARIENT ANALYSIS

In [ ]:
# GET THE TOP 10 AREAS BY PRODUCTION IN TONES IN BAR CHART'
grouped_crop_DF = grouped_crop_Dataframe.groupby('Area', as_index=False).sum()
top_10_areas = grouped_crop_DF.nlargest(10, 'Production in Tones')
plt.figure(figsize=(12, 6))
sns.barplot(x="Area", y="Production in Tones", data=top_10_areas, palette="viridis")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.title("Top 10 Areas by Production in Tones")
plt.xlabel("Area")
plt.ylabel("Production in Tones")
plt.tight_layout()
plt.show()

In [ ]:
# GET THE TOP 10 ITEMS BY PRODUCTION IN TONES IN BAR CHART'
grouped_crop_DF = grouped_crop_Dataframe.groupby('Item', as_index=False).sum()
top_10_areas = grouped_crop_DF.nlargest(10, 'Production in Tones')
plt.figure(figsize=(12, 6))
sns.barplot(x="Area", y="Production in Tones", data=top_10_areas, palette="viridis")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.title("Top 10 Item by Production in Tones")
plt.xlabel("Area")
plt.ylabel("Production in Tones")
plt.tight_layout()
plt.show()

MULTI-VARIENT ANALYSIS

In [ ]:
# GET THE TOP 10 AREA FOR EACH YEAR
grouped_crop_DF = grouped_crop_Dataframe.groupby(['Year', 'Area'], as_index=False).sum()
top_10_areas_per_year = grouped_crop_DF.groupby('Year').apply(lambda x: x.nlargest(10, 'Production in Tones')).reset_index(drop=True)
plt.figure(figsize=(12, 6))
sns.barplot(x="Area", y="Production in Tones", hue="Year", data=top_10_areas_per_year, palette="viridis")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.title("Top 10 Area by Production in Tones for Each Year")
plt.xlabel("Area")
plt.ylabel("Production in Tones")
plt.legend(title='Year')
plt.tight_layout()
plt.show()

In [ ]:
# GET THE TOP 10 ITEM FOR EACH YEAR
grouped_crop_DF = grouped_crop_Dataframe.groupby(['Year', 'Item'], as_index=False).sum()
top_10_areas_per_year = grouped_crop_DF.groupby('Year').apply(lambda x: x.nlargest(10, 'Production in Tones')).reset_index(drop=True)
plt.figure(figsize=(12, 6))
sns.barplot(x="Item", y="Production in Tones", hue="Year", data=top_10_areas_per_year, palette="viridis")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.title("Top 10 Item by Production in Tones for Each Year")
plt.xlabel("Items")
plt.ylabel("Production in Tones")
plt.legend(title='Year')
plt.tight_layout()
plt.show()

                                              MODEL TRAINING AND PREPROCESSING

In [540]:
#IMPORTING THE LIBRARIES FOR MODEL TRAINING AND SPLITING
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [541]:
# CONVERT PRODUCTION IN TONNES TO KILOTONNES (DIVIDE BY 1000) FOR TRAINING THE GOOD MODEL
grouped_crop_Dataframe["Production in Kilotonnes"] = grouped_crop_Dataframe["Production in Tones"] / 1000

# CONVERT THE COLUMN TO FLOAT (IF IT'S NOT ALREADY) AND ROUND IT TO 3 DECIMAL PLACES
grouped_crop_Dataframe["Production in Kilotonnes"] = grouped_crop_Dataframe["Production in Kilotonnes"].astype(float)

# ROUND THE VALUES TO 3 DECIMAL POINTS
grouped_crop_Dataframe["Production in Kilotonnes"] = grouped_crop_Dataframe["Production in Kilotonnes"].round(3)

In [ ]:
#DISPLAY THE DATA AFTER CONVERT THE PRODUCTION IN TONES TO KILOTONNES
grouped_crop_Dataframe.head()

In [543]:
#CHOOSING THE COLUMNS FOR TRAINING AND TESTING THE MODEL X:FEATURES AND Y: TARGET
x=grouped_crop_Dataframe[['Area_Harvested_in_Hectares','Yield_Value in kg/ha(or)mg/Ar','Producing Animals/Slaughtered_Value in An','Laying_Value in An','Yield/Carcass Weight_Value in g/An','Milk Animals_Value in An']]
y=grouped_crop_Dataframe['Production in Kilotonnes']

In [ ]:
#REDUCE THE SIZE OF X BUT NOT USED IN MODEL TRAINING BECAUSE IT NOT GOOD WHILE TRAINING
X = grouped_crop_Dataframe[['Area_Harvested_in_Hectares','Yield_Value in kg/ha(or)mg/Ar',
                            'Producing Animals/Slaughtered_Value in An','Laying_Value in An',
                            'Yield/Carcass Weight_Value in g/An','Milk Animals_Value in An']]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled.head()


In [545]:
#SPLITING THE FEATURE AND TARGET FOR THE TRAINING MODEL
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
# Check the range of y_test
print(f"Range of y_test: min={y_test.min()}, max={y_test.max()}")

In [547]:
#CREATING THE OBJECT FOR CALLING THE LINEAR REG MODDEL
linear_model=LinearRegression()

In [548]:
#TRAING THE MODEL BY TRAING DATA
linear_model.fit(x_train,y_train)

LinearRegression()

In [549]:
#CHECKING THE PREDICTION FOR THE TRAINED MODEL BY GIVING TESTING DATA
y_predict=linear_model.predict(x_test)

In [ ]:
#DISPLAY THE PREDICTED OUTPUT
y_predict

In [ ]:
#CHECKING THE SCORE FOR THE TRAINED MODEL
linear_model_r2 = linear_model.score(x_test, y_test)  
print(f"R square: {linear_model_r2:.2f}")


In [ ]:
#CHECKING THE ERROR FOR THE TRAINED MODEL
linear_model_mse = mean_squared_error(y_test, y_predict)
print(f"Mean Squared Error: {linear_model_mse:.2f}")

In [553]:
#CREATING THE DATAFRAME FOR TESTING THE MODEL WITH NEW DATA SET
data_dict = {
    
    "Area_Harvested_in_Hectares": [29203.0],
    "Yield_Value in kg/ha(or)mg/Ar": [1308.3],
    "Producing Animals/Slaughtered_Value in An": [0.0],
    "Laying_Value in An": [0.0],
    "Yield/Carcass Weight_Value in g/An": [0.0],
    "Milk Animals_Value in An": [0.0]
}
data_df = pd.DataFrame(data_dict)



In [ ]:
#PREDICT THE OUTPUT WITH NEW DATA SET
y_predict_test = linear_model.predict(data_df)
print(y_predict_test)

In [555]:
#TRAINED MODEL WITH LINEAR REG SCORE IS LOW SO WE GO FOR RIDGE REG
from sklearn.linear_model import Ridge


In [556]:
#CREATING THE OBJECT FOR CALLING THE LINEAR RIDGE MODDEL
ridge_model = Ridge(alpha=1.0)

In [557]:
#TRAING THE MODEL BY TRAING DATA
ridge_model.fit(x_train, y_train)

Ridge()

In [ ]:
#CHECKING THE PREDICTION FOR THE TRAINED RIDGE MODEL BY GIVING TESTING DATA
y1_pred = ridge_model.predict(x_test)
y1_pred

In [ ]:
#CHECKING THE SCORE FOR THE TRAINED RIDGE MODEL
ridge_model_r2= ridge_model.score(x_test, y_test)  
print(f"R square: {ridge_model_r2:.2f}")


In [560]:
#TRAINED MODEL WITH LASSO REG SCORE IS LOW SO WE GO FOR RIDGE REG
from sklearn.linear_model import Lasso

In [561]:
#CREATING THE OBJECT FOR CALLING THE LINEAR LASSO MODDEL
lasso_model = Lasso(alpha=1.0)

In [562]:
#TRAING THE MODEL BY TRAING DATA
lasso_model.fit(x_train, y_train)

Lasso()

In [ ]:
#CHECKING THE PREDICTION FOR THE TRAINED LASSO MODEL BY GIVING TESTING DATA
y2_pred = lasso_model.predict(x_test)
y2_pred

In [ ]:
#CHECKING THE SCORE FOR THE TRAINED LASSO REG MODEL
lasso_model_r2 = ridge_model.score(x_test, y_test)  
print(f"R2 Square: {lasso_model_r2:.2f}")

In [ ]:
#PREDICT THE OUTPUT WITH NEW DATA SET
y3_predict_test = lasso_model.predict(data_df)
print(y3_predict_test)

In [566]:
#SCORE IS LOW FOR LASSO REG WE GO FOR RANDOM FOREST REGRESSOR
from sklearn.ensemble import RandomForestRegressor


In [567]:
#CREATING THE OBJECT FOR CALLING THE RANDOM FOREST REG MODDEL
rf_model = RandomForestRegressor(n_estimators=100, random_state=42) 

In [568]:
#TRAING THE RANDOM FOREST REG MODEL BY TRAING DATA
rf_model.fit(x_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
##CHECKING THE PREDICTION FOR THE TRAINED RF MODEL BY GIVING TESTING DATA
y3_pred = rf_model.predict(x_test)
y3_pred

In [ ]:
#CHECKING THE SCORE FOR THE TRAINED RF REG MODEL
rf_model_r2 = rf_model.score(x_test, y_test) 
print(f"R Square: {rf_model_r2:.2f}")

In [ ]:
#CHECKING THE ERROR FOR THE TRAINED MODEL
rf_model_mse = mean_squared_error(y_test, y_predict)
print(f"Mean Squared Error: {rf_model_mse:.2f}")

In [572]:
#PREDICT THE OUTPUT WITH NEW DATA SET
y3_predict_test = rf_model.predict(data_df)
print(y3_predict_test)

[38.3989]


In [573]:
#IMPORT JOBLIB TO SAVE THE TRAINED GOOD MODEL
import joblib

In [ ]:


# SAVE THE GOOD TRAINED RANDOM FOREST REG MODEL
joblib.dump(rf_model, r'c:\Users\arune\OneDrive\Desktop\CROP PRODUCTION PREDICTION.pkl')


In [495]:
#RANDOM FOREST REG TRAINED MODEL SCORE IS GOOD , EVENTHOUGH TRAINING THE MODEL BY XGBOOST FOR CHECKING THE SCORE
import xgboost as xgb

In [496]:
#CREATING THE OBJECT FOR CALLING THE XG BOOST REG MODDEL
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1, max_depth=5, alpha=10, n_estimators=100)

In [497]:
##TRAING THE XG BOOST REG MODEL BY TRAING DATA
xg_reg.fit(x_train, y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [ ]:
#CHECKING THE PREDICTION FOR THE TRAINED XG BOOST MODEL BY GIVING TESTING DATA
y4_pred = xg_reg.predict(x_test)
y4_pred

In [ ]:
#CHECKING THE ERROR FOR THE TRAINED XG BOOST REG MODEL
xgboost_model_mse = mean_squared_error(y_test, y4_pred)
print(f"Mean Squared Error: {xgboost_model_mse:.2f}")

In [ ]:
#CHECKING THE ERROR FOR THE TRAINED XG BOOST MODEL
xgboost_model_r2 = xg_reg.score(x_test, y_test) 
print(f"R Square: {xgboost_model_r2:.2f}")